## 1. Data Loading

In [1]:
import pickle

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file
print("Loading datafile...")
with open("mon_standard.pkl", 'rb') as fi:
    data = pickle.load(fi)

X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
y = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]

# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.
    for sample in data[i]:
        size_seq = []
        time_seq = []
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
        X1.append(time_seq)
        X2.append(size_seq)
        y.append(label)
size = len(y)

print(f'Total samples: {size}') # Output: 19000


Loading datafile...
Total samples: 19000


## 2. Feature Engineering

In [2]:
import numpy as np
from itertools import groupby

# Initialize a list to store all features
features = []

# Feature extraction
for i in range(len(X2)):
    size_seq = X2[i]
    time_seq = X1[i]
    
    # 1. Number of incoming packets
    num_incoming_packets = sum(1 for size in size_seq if size < 0)
    
    # 2. Total number of packets
    num_total_packets = len(size_seq)
    
    # 3. Number of outgoing packets
    num_outgoing_packets = sum(1 for size in size_seq if size > 0)
    
    # 4. Proportion of outgoing packets
    ratio_outgoing = num_outgoing_packets / num_total_packets if num_total_packets > 0 else 0
    
    # 5. Proportion of incoming packets
    ratio_incoming = num_incoming_packets / num_total_packets if num_total_packets > 0 else 0
    
    # 6. Proportion of incoming packets in the first 30 packets
    incoming_first_30 = sum(1 for size in size_seq[:30] if size < 0) / 30 if num_total_packets >= 30 else 0
    
    # 7. Proportion of outgoing packets in the first 30 packets
    outgoing_first_30 = sum(1 for size in size_seq[:30] if size > 0) / 30 if num_total_packets >= 30 else 0
    
    # 8. Maximum number of packets in outgoing bursts
    burst_outgoing_max = max(len(list(g)) for k, g in groupby(size_seq) if k > 0) if num_outgoing_packets > 0 else 0
    
    # 9. Standard deviation of outgoing burst sizes
    outgoing_burst_std = np.std([len(list(g)) for k, g in groupby(size_seq) if k > 0])
    
    # 10. Average size of outgoing bursts
    outgoing_burst_avg = np.mean([len(list(g)) for k, g in groupby(size_seq) if k > 0])
    
    # 11. Number of outgoing bursts
    num_outgoing_bursts = len([1 for k, g in groupby(size_seq) if k > 0])
    
    # 12. Number of incoming bursts
    num_incoming_bursts = len([1 for k, g in groupby(size_seq) if k < 0])
    
    # 13. Average number of incoming packets per second
    incoming_per_second_avg = num_incoming_packets / (len(time_seq) / 60) if len(time_seq) > 0 else 0

    # 14. Total transmission time
    total_time = sum(time_seq)

    # 15. Average time interval in the last 5 seconds
    last_5_seconds_time_intervals = np.mean([time_seq[i+1] - time_seq[i] for i in range(len(time_seq)-1) if time_seq[i] > 0]) if len(time_seq) > 1 else 0
    
    # 16. Sum of packet sizes
    sum_packets = sum(size_seq)
    
    # 17. Packets per second
    packets_per_second = len(size_seq) / sum(time_seq) if sum(time_seq) > 0 else 0
    
    # 18. Mean packets per second
    packets_per_second_values = [1 / t for t in time_seq if t > 0]
    
    # 19. mean of the packets per second values
    mean_packets_per_second = np.mean(packets_per_second_values) if packets_per_second_values else 0
    
    # 20. Standard deviation of packets per second
    std_packets_per_second = np.std(packets_per_second_values) if packets_per_second_values else 0
    
    # 21. Transmission time Q1 (25th percentile)
    transmission_time_Q1 = np.percentile(time_seq, 25) if len(time_seq) > 0 else 0
    
    # 22. Transmission time Q2 (median)
    transmission_time_Q2 = np.percentile(time_seq, 50) if len(time_seq) > 0 else 0
    
    # 23. Transmission time Q3 (75th percentile)
    transmission_time_Q3 = np.percentile(time_seq, 75) if len(time_seq) > 0 else 0
    
    # 24. Transmission time Q4 (maximum)
    transmission_time_Q4 = np.percentile(time_seq, 100) if len(time_seq) > 0 else 0
    
    # 25. Average ordering of outgoing packets
    outgoing_order = [i for i, size in enumerate(size_seq) if size > 0]
    average_outgoing_ordering = np.mean(outgoing_order) if outgoing_order else 0
    
    # 26. Standard deviation of outgoing packet ordering
    std_dev_outgoing_ordering = np.std(outgoing_order) if outgoing_order else 0
    
    # Add all features to the list
    features.append([
        num_incoming_packets, num_total_packets, num_outgoing_packets, ratio_outgoing,
        ratio_incoming, incoming_first_30, outgoing_first_30, burst_outgoing_max,
        outgoing_burst_std, outgoing_burst_avg, num_outgoing_bursts,
        num_incoming_bursts, incoming_per_second_avg,  total_time,
        last_5_seconds_time_intervals, 
        sum_packets, packets_per_second, mean_packets_per_second, std_packets_per_second,
        transmission_time_Q1, transmission_time_Q2, transmission_time_Q3, transmission_time_Q4,
        average_outgoing_ordering, std_dev_outgoing_ordering
    ])

# Convert the feature list to a numpy array
features = np.array(features)

# Labels for each sample
y = np.array(y)


## 3. Data Splitting and Scaling

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# data split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42, stratify=y)

# scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 4-1. Train the Extra Tree model

In [4]:
from sklearn.ensemble import ExtraTreesClassifier

et_model = ExtraTreesClassifier(
    n_estimators=110,
    max_depth=35,
    max_features=0.5,
    random_state=42
)

et_model.fit(X_train, y_train)

ExtraTreesClassifier(max_depth=35, max_features=0.5, n_estimators=110,
                     random_state=42)

## 4-2. Test and Evaluate the Extra Tree model

In [6]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix, precision_score, precision_recall_curve, roc_curve, recall_score, f1_score, classification_report

y_pred = et_model.predict(X_test)

# Calculate Accuracy, Precison, Recall, F1 Score 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate Confusion Matrix, ROC AUC, PR AUC
conf_matrix = confusion_matrix(y_test, y_pred)

# Classification Report
class_report = classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(95)])

# Results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Weighted): {precXision:.4f}")
print(f"Recall (Weighted): {recall:.4f}")
print(f"F1 Score (Weighted): {f1:.4f}")

print(f"\nConfusion Matrix\n{conf_matrix}")
print("\nClassification Report:\n", class_report)

Accuracy: 0.8084
Precision (Weighted): 0.8111
Recall (Weighted): 0.8084
F1 Score (Weighted): 0.8063

Confusion Matrix
[[20  0  0 ...  0  0  0]
 [ 0 34  0 ...  0  0  0]
 [ 0  0 36 ...  0  0  0]
 ...
 [ 0  0  0 ... 27  0  1]
 [ 0  0  0 ...  0 39  0]
 [ 0  0  0 ...  0  0 28]]

Classification Report:
               precision    recall  f1-score   support

     Class 0       0.91      0.50      0.65        40
     Class 1       0.85      0.85      0.85        40
     Class 2       0.92      0.90      0.91        40
     Class 3       0.86      0.90      0.88        40
     Class 4       0.74      0.88      0.80        40
     Class 5       0.87      0.85      0.86        40
     Class 6       0.78      0.88      0.82        40
     Class 7       0.89      0.97      0.93        40
     Class 8       0.84      0.80      0.82        40
     Class 9       0.75      0.82      0.79        40
    Class 10       0.94      0.80      0.86        40
    Class 11       0.91      0.78      0.84        4

## 5-1. Train the XGBoost model

In [7]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    n_estimators=110,
    max_depth=35,
    learning_rate=0.1,  
    subsample=0.8, 
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=35, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=110, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

## 5-2. Test and Evaluate the XGBoost model

In [8]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix, precision_score, precision_recall_curve, roc_curve, recall_score, f1_score, classification_report

y_pred = xgb_model.predict(X_test)

# Calculate Accuracy, Precison, Recall, F1 Score 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate Confusion Matrix, ROC AUC, PR AUC
conf_matrix = confusion_matrix(y_test, y_pred)

# Classification Report
class_report = classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(95)])

# Results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Weighted): {precision:.4f}")
print(f"Recall (Weighted): {recall:.4f}")
print(f"F1 Score (Weighted): {f1:.4f}")

print(f"\nConfusion Matrix\n{conf_matrix}")
print("\nClassification Report:\n", class_report)

Accuracy: 0.7874
Precision (Weighted): 0.7900
Recall (Weighted): 0.7874
F1 Score (Weighted): 0.7856

Confusion Matrix
[[23  0  0 ...  0  0  1]
 [ 0 35  0 ...  0  0  0]
 [ 0  0 34 ...  0  0  0]
 ...
 [ 0  0  0 ... 24  0  1]
 [ 0  0  0 ...  0 39  0]
 [ 1  0  0 ...  0  0 28]]

Classification Report:
               precision    recall  f1-score   support

     Class 0       0.64      0.57      0.61        40
     Class 1       0.83      0.88      0.85        40
     Class 2       0.94      0.85      0.89        40
     Class 3       0.90      0.88      0.89        40
     Class 4       0.74      0.88      0.80        40
     Class 5       0.75      0.75      0.75        40
     Class 6       0.79      0.82      0.80        40
     Class 7       0.80      0.90      0.85        40
     Class 8       0.81      0.75      0.78        40
     Class 9       0.76      0.80      0.78        40
    Class 10       0.88      0.70      0.78        40
    Class 11       0.91      0.75      0.82        4

## 6-1. Train the Random Forest model

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=110,
    max_depth=35,
    max_features='sqrt',
    random_state=42
)

rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=35, n_estimators=110, random_state=42)

## 6-2. Test and Evaluate the Random Forest model

In [10]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix, precision_score, precision_recall_curve, roc_curve, recall_score, f1_score, classification_report

y_pred = rf_model.predict(X_test)

# Calculate Accuracy, Precison, Recall, F1 Score 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate Confusion Matrix, ROC AUC, PR AUC
conf_matrix = confusion_matrix(y_test, y_pred)

# Classification Report
class_report = classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(95)])

# Results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Weighted): {precision:.4f}")
print(f"Recall (Weighted): {recall:.4f}")
print(f"F1 Score (Weighted): {f1:.4f}")

print(f"\nConfusion Matrix\n{conf_matrix}")
print("\nClassification Report:\n", class_report)

Accuracy: 0.7982
Precision (Weighted): 0.8003
Recall (Weighted): 0.7982
F1 Score (Weighted): 0.7960

Confusion Matrix
[[22  0  0 ...  0  0  0]
 [ 0 33  0 ...  0  0  0]
 [ 0  0 37 ...  0  0  0]
 ...
 [ 0  0  0 ... 26  0  1]
 [ 0  0  0 ...  0 39  0]
 [ 0  0  0 ...  0  0 28]]

Classification Report:
               precision    recall  f1-score   support

     Class 0       0.81      0.55      0.66        40
     Class 1       0.85      0.82      0.84        40
     Class 2       0.93      0.93      0.93        40
     Class 3       0.85      0.88      0.86        40
     Class 4       0.74      0.85      0.79        40
     Class 5       0.87      0.85      0.86        40
     Class 6       0.74      0.85      0.79        40
     Class 7       0.86      0.93      0.89        40
     Class 8       0.88      0.88      0.88        40
     Class 9       0.74      0.85      0.79        40
    Class 10       0.94      0.82      0.88        40
    Class 11       0.83      0.75      0.79        4